In [41]:
import pandas as pd


In [42]:
import re
import numpy as np

In [43]:
df = pd.read_csv('critic_reviews.csv', index_col = 0).drop('index', axis = 1)

In [44]:
df

,movie_name,critics and company,date,review,score,decision
0,chicken_run,"['Dejan Jotanovic', 'Bitch Media']","March 15, 2021","Chicken Run, if anything, offers an alternativ...",[],fresh
1,chicken_run,"['Rebecca Hawkes', 'Daily Telegraph (UK)']","July 20, 2015","Its plump chickens maintain that strange, hapt...",Original Score: 5/5,fresh
2,chicken_run,"['Anthony Lane', 'New Yorker']","July 20, 2015","Even without Wallace and Gromit, most of the t...",[],fresh
3,chicken_run,"['Debra Jo Immergut', 'Wall Street Journal']","July 20, 2015","It coaxes you to laugh and cry, flatters your ...",[],fresh
4,chicken_run,"['Peter Bradshaw', 'Guardian']","August 24, 2014",Chicken Run is a charmingly conceived entertai...,[],fresh
...,...,...,...,...,...,...
41252,mank,"['David Sims', 'The Atlantic']","December 4, 2020","Mank is a tale of triumph, the kind that's bee...",[],fresh
41253,mank,"['A.O. Scott', 'New York Times']","December 3, 2020","A worthy, eminently watchable entry in the ann...",[],fresh
41254,mank,"['James Berardinelli', 'ReelViews']","December 3, 2020",Although Fincher's attention to detail results...,Original Score: 3/4,fresh
41255,mank,"['Nell Minow', 'Movie Mom']","December 3, 2020",It is pure pleasure to see a film that respect...,Original Score: A-,fresh


In [45]:
df_summary = pd.DataFrame({'movie': (df['movie_name']).unique()})

In [46]:
#tf_idf_scores = pd.DataFrame()
movie_summaries = []
for i in df_summary['movie']:
    tf_idf_df = pd.DataFrame({'DOCUMENT':df.loc[df['movie_name'] == i]['review']})
    
    tf_idf_df['DOCUMENT'] = tf_idf_df['DOCUMENT'].str.lower()
    words = tf_idf_df['DOCUMENT'].str.replace('[^ a-z0-9]', ' ', regex = True).str.replace(' +', ' ').str.split().sum()
    for w in pd.Series(words).unique():
        re_pat = '\\b%s\\b' % w
        #print(tf_idf_df['DOCUMENT'])
        tf = tf_idf_df['DOCUMENT'].str.count(re_pat) / (tf_idf_df['DOCUMENT'].str.count(' ') + 1)
        idf = np.log(len(tf_idf_df['DOCUMENT']) / tf_idf_df['DOCUMENT'].str.contains(re_pat).sum())
        tf_idf_df[w] = tf * idf
    #tf_idf_scores = pd.concat([tf_idf_scores, tf_idf_df])
    average = tf_idf_df.sum(axis = 1).mean()
    summary_series = tf_idf_df.set_index('DOCUMENT').sum(axis = 1)
    summary_text = summary_series[tf_idf_df.set_index('DOCUMENT').sum(axis = 1) >=  1.2 * average].index
    if len(summary_text) == 0:
        summary_text = summary_series[tf_idf_df.set_index('DOCUMENT').sum(axis = 1) >= average].index
    my_summary = ''
    for summary in summary_text:
        my_summary += summary + ' '
    movie_summaries.append(my_summary)
df_summary['summary'] = movie_summaries

<ipython-input-46-88431d0d21fd>:12: RuntimeWarning: divide by zero encountered in long_scalars
  idf = np.log(len(tf_idf_df['DOCUMENT']) / tf_idf_df['DOCUMENT'].str.contains(re_pat).sum())


In [47]:

df_summary.to_csv('tf_idf_summarized.csv')

In [48]:
df_summary = pd.read_csv('tf_idf_summarized.csv', index_col = 0)

In [50]:
df_summary.dropna()

,movie,summary
0,chicken_run,chicken run isn't just chock-full of reference...
1,the_taste_of_others,"a worldly-wise, compassionate and charmingly a..."
2,yi_yi,"keenly observed. chances are, you'll watch mos..."
3,memento,i can't remember when a movie has seemed so cl...
4,the_life_and_times_of_hank_greenberg,it's an adroit manipulation of interviews and ...
...,...,...
2095,emma_2020,"while the story's sturdy, familiar structure r..."
2096,yes_god_yes,karen maine's script confronts catholicism's b...
2097,blood_on_her_name,anchored by a magnetic performance from bethan...
2098,sylvies_love,"every element of ashe's film is swoon-worthy, ..."


# Used for across genre comparisons / Across time comparisons

In [51]:
all_words = df_summary['summary'].str.replace('[^ a-z0-9]', ' ', regex = True).str.replace(' +', ' ').str.split().sum()

In [52]:
all_words = pd.Series(all_words)

In [53]:
tfidf_2 = pd.DataFrame([], index=df_summary.index)  # dataframe of documents
tf_denom = (df_summary.summary.str.count(' ') + 1)
for w in all_words.value_counts().index:
    # imperfect pattern match for speed
    re_pat = ' %s ' % w
    tf = df_summary.summary.str.count(re_pat) / tf_denom
    idf = np.log(len(df_summary) / df_summary.summary.str.contains(re_pat).sum())
    tfidf_2[w] =  tf * idf

<ipython-input-53-0dd735eb0826>:7: RuntimeWarning: divide by zero encountered in long_scalars
  idf = np.log(len(df_summary) / df_summary.summary.str.contains(re_pat).sum())


In [54]:
tfidf_2.dropna(axis=1, how = 'all', inplace = True)

In [55]:
def five_largest(row):
    
    return list(row.index[row.argsort()][-5:])

In [56]:
keywords = tfidf_2.apply(five_largest, axis=1)

In [57]:
keywords_df = pd.concat([
    df_summary,
    keywords
], axis=1)

In [58]:
keywords_df

,movie,summary,0
0,chicken_run,chicken run isn't just chock-full of reference...,"[secret, missing, references, sadly, ingredient]"
1,the_taste_of_others,"a worldly-wise, compassionate and charmingly a...","[and, acted, compassionate, charmingly, gallic]"
2,yi_yi,"keenly observed. chances are, you'll watch mos...","[watch, miss, smile, indulgent, chances]"
3,memento,i can't remember when a movie has seemed so cl...,"[seemed, strangely, remember, slyly, nolan]"
4,the_life_and_times_of_hank_greenberg,it's an adroit manipulation of interviews and ...,"[hank, athlete, baseball, aviva, greenberg]"
...,...,...,...
2095,emma_2020,"while the story's sturdy, familiar structure r...","[particularly, version, inspired, structure, s..."
2096,yes_god_yes,karen maine's script confronts catholicism's b...,"[with, script, satirical, hypocrisy, confronts]"
2097,blood_on_her_name,anchored by a magnetic performance from bethan...,"[bethany, 83, outwit, southern, lind]"
2098,sylvies_love,"every element of ashe's film is swoon-worthy, ...","[costumes, phoenix, lush, declan, fabrice]"


In [88]:
df_by_movie = pd.DataFrame({'movie': df_summary['movie']})
char_words_list = []
for i in df_summary['movie']:
    tf_idf_df = pd.DataFrame({'DOCUMENT':df.loc[df['movie_name'] == i]['review']})
    tf_idf_df['DOCUMENT'] = tf_idf_df['DOCUMENT'].str.lower()
    words = tf_idf_df['DOCUMENT'].str.replace('[^ a-z0-9]', ' ', regex = True).str.replace(' +', ' ').str.split().sum()
    for w in pd.Series(words).unique():
        re_pat = '\\b%s\\b' % w
        #print(tf_idf_df['DOCUMENT'])
        tf = tf_idf_df['DOCUMENT'].str.count(re_pat) / (tf_idf_df['DOCUMENT'].str.count(' ') + 1)
        idf = np.log(len(tf_idf_df['DOCUMENT']) / tf_idf_df['DOCUMENT'].str.contains(re_pat).sum())
        tf_idf_df[w] = tf * idf
    char_words = list(tf_idf_df.mean()[1:].sort_values(ascending = False).head(5).index)
    char_words_list.append(char_words)
df_by_movie['char_words'] = char_words_list

<ipython-input-88-a4de161b2fca>:11: RuntimeWarning: divide by zero encountered in long_scalars
  idf = np.log(len(tf_idf_df['DOCUMENT']) / tf_idf_df['DOCUMENT'].str.contains(re_pat).sum())


In [89]:
df_by_movie

,movie,char_words
0,chicken_run,"[this, run, the, it, animation]"
1,the_taste_of_others,"[delight, civilized, a, the, with]"
2,yi_yi,"[keenly, observed, masterwork, humanistic, imp..."
3,memento,"[to, a, nolan, film, christopher]"
4,the_life_and_times_of_hank_greenberg,"[as, the, of, subject, its]"
...,...,...
2095,emma_2020,"[adaptation, a, it, is, this]"
2096,yes_god_yes,"[in, and, a, s, an]"
2097,blood_on_her_name,"[confident, exceptionally, debut, an, unexpected]"
2098,sylvies_love,"[s, love, her, on, for]"
